In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Input, LSTM, Dropout, Dense, concatenate
from keras.utils import to_categorical
from keras.models import Model

In [2]:
df = pd.read_csv('./Data/merged_football_data.csv')

In [3]:
df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")

In [4]:
df["H_Team"] = df["HomeTeam"]
df["A_Team"] = df["AwayTeam"]
df

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HF,AF,HC,AC,HY,AY,HR,AR,H_Team,A_Team
0,2011-08-13,Man United,Leicester,2,1,H,1,0,H,A Marriner,...,11,8,2,5,2,1,0,0,Man United,Leicester
1,2011-08-13,Bournemouth,Cardiff,2,0,H,1,0,H,K Friend,...,11,9,7,4,1,1,0,0,Bournemouth,Cardiff
2,2011-08-13,Fulham,Crystal Palace,0,2,A,0,1,A,M Dean,...,9,11,5,5,1,2,0,0,Fulham,Crystal Palace
3,2011-08-13,Huddersfield,Chelsea,0,3,A,0,2,A,C Kavanagh,...,9,8,2,5,2,1,0,0,Huddersfield,Chelsea
4,2011-08-13,Newcastle,Tottenham,1,2,A,1,2,A,M Atkinson,...,11,12,3,5,2,2,0,0,Newcastle,Tottenham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3625,2023-05-28,Swansea,Fulham,0,3,A,0,1,A,L Mason,...,12,9,8,0,2,1,0,0,Swansea,Fulham
3626,2023-05-28,Tottenham,Sunderland,1,0,H,0,0,D,A Marriner,...,6,12,14,1,1,3,0,1,Tottenham,Sunderland
3627,2023-05-28,West Brom,Man United,5,5,D,1,3,A,M Oliver,...,10,6,3,5,0,1,0,0,West Brom,Man United
3628,2023-05-28,West Ham,Reading,4,2,H,2,0,H,M Dean,...,14,8,6,4,2,1,0,0,West Ham,Reading


In [5]:
df['HomeTeam'] = df['HomeTeam'].astype('category').cat.codes
df['AwayTeam'] = df['AwayTeam'].astype('category').cat.codes

In [6]:
df["day_code"] = df["Date"].dt.dayofweek
df["target"] = (df["FTR"] == "H").astype("int")
df.loc[df["FTR"] == "D","target"] = 2
df

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HC,AC,HY,AY,HR,AR,H_Team,A_Team,day_code,target
0,2011-08-13,19,16,2,1,H,1,0,H,A Marriner,...,2,5,2,1,0,0,Man United,Leicester,5,1
1,2011-08-13,4,8,2,0,H,1,0,H,K Friend,...,7,4,1,1,0,0,Bournemouth,Cardiff,5,1
2,2011-08-13,12,10,0,2,A,0,1,A,M Dean,...,5,5,1,2,0,0,Fulham,Crystal Palace,5,0
3,2011-08-13,13,9,0,3,A,0,2,A,C Kavanagh,...,2,5,2,1,0,0,Huddersfield,Chelsea,5,0
4,2011-08-13,21,30,1,2,A,1,2,A,M Atkinson,...,3,5,2,2,0,0,Newcastle,Tottenham,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3625,2023-05-28,29,12,0,3,A,0,1,A,L Mason,...,8,0,2,1,0,0,Swansea,Fulham,6,0
3626,2023-05-28,30,28,1,0,H,0,0,D,A Marriner,...,14,1,1,3,0,1,Tottenham,Sunderland,6,1
3627,2023-05-28,32,19,5,5,D,1,3,A,M Oliver,...,3,5,0,1,0,0,West Brom,Man United,6,2
3628,2023-05-28,33,25,4,2,H,2,0,H,M Dean,...,6,4,2,1,0,0,West Ham,Reading,6,1


In [7]:
df_matches = df.groupby("H_Team")
group = df_matches.get_group("Arsenal")
group

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HC,AC,HY,AY,HR,AR,H_Team,A_Team,day_code,target
7,2011-08-14,0,18,0,2,A,0,1,A,M Oliver,...,2,9,2,2,0,0,Arsenal,Man City,6,0
20,2011-08-27,0,33,3,1,H,1,1,D,G Scott,...,10,2,1,3,0,0,Arsenal,West Ham,5,1
58,2011-09-26,0,11,2,0,H,0,0,D,J Moss,...,5,9,2,1,0,0,Arsenal,Everton,0,1
60,2011-10-01,0,31,2,0,H,0,0,D,A Taylor,...,6,6,2,2,0,0,Arsenal,Watford,5,1
89,2011-10-29,0,16,3,1,H,1,1,D,C Kavanagh,...,6,4,2,2,0,0,Arsenal,Leicester,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3546,2023-04-15,0,25,4,1,H,1,0,H,C Foy,...,11,3,0,1,0,0,Arsenal,Reading,5,1
3566,2023-04-25,0,22,3,1,H,0,0,D,M Jones,...,9,3,1,2,0,0,Arsenal,Norwich,1,1
3573,2023-04-27,0,11,0,0,D,0,0,D,N Swarbrick,...,8,1,2,3,0,0,Arsenal,Everton,3,2
3593,2023-05-07,0,19,1,1,D,1,1,D,P Dowd,...,7,2,3,5,0,0,Arsenal,Man United,6,2


In [8]:
def rolling_averages(group, cols,new_cols):
  group = group.sort_values("Date")
  rolling_stats = group[cols].rolling(3,closed="left").mean()
  group[new_cols] = rolling_stats
  group = group.dropna(subset=new_cols)
  return group

In [9]:
cols = ["HS","AS","HST","AST","HF","AF","HC","AC","HY","AY","HR","AR"]
new_cols = [f"{c}_rolling" for c in cols]

In [10]:
rolling_averages(group,cols,new_cols)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HST_rolling,AST_rolling,HF_rolling,AF_rolling,HC_rolling,AC_rolling,HY_rolling,AY_rolling,HR_rolling,AR_rolling
60,2011-10-01,0,31,2,0,H,0,0,D,A Taylor,...,6.000000,6.333333,14.666667,13.000000,5.666667,6.666667,1.666667,2.000000,0.0,0.0
89,2011-10-29,0,16,3,1,H,1,1,D,C Kavanagh,...,5.666667,5.000000,14.666667,14.000000,7.000000,5.666667,1.666667,2.000000,0.0,0.0
100,2011-11-05,0,17,1,1,D,0,0,D,A Marriner,...,4.333333,4.000000,12.666667,13.000000,5.666667,6.333333,2.000000,1.666667,0.0,0.0
116,2011-11-19,0,35,1,1,D,0,1,A,S Attwell,...,4.000000,3.333333,9.333333,11.333333,5.666667,6.000000,1.666667,1.666667,0.0,0.0
137,2011-12-04,0,30,4,2,H,1,2,A,M Dean,...,4.333333,3.666667,8.666667,11.000000,7.333333,4.666667,1.666667,1.666667,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3546,2023-04-15,0,25,4,1,H,1,0,H,C Foy,...,9.666667,3.333333,6.333333,12.000000,10.333333,3.333333,1.333333,2.000000,0.0,0.0
3566,2023-04-25,0,22,3,1,H,0,0,D,M Jones,...,12.333333,2.333333,6.333333,13.000000,11.333333,2.333333,0.333333,2.000000,0.0,0.0
3573,2023-04-27,0,11,0,0,D,0,0,D,N Swarbrick,...,11.666667,3.333333,8.000000,12.000000,10.666667,3.000000,0.666667,1.666667,0.0,0.0
3593,2023-05-07,0,19,1,1,D,1,1,D,P Dowd,...,9.000000,4.000000,9.333333,12.000000,9.333333,2.333333,1.000000,2.000000,0.0,0.0


In [11]:
matches_rolling = df.groupby("H_Team").apply(lambda x:rolling_averages(x,cols,new_cols))

In [12]:
matches_rolling

Date  HomeTeam  AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
H_Team                                                                        
Arsenal 60   2011-10-01         0        31     2     0   H     0     0   D   
        89   2011-10-29         0        16     3     1   H     1     1   D   
        100  2011-11-05         0        17     1     1   D     0     0   D   
        116  2011-11-19         0        35     1     1   D     0     1   A   
        137  2011-12-04         0        30     4     2   H     1     2   A   
...                 ...       ...       ...   ...   ...  ..   ...   ...  ..   
Wolves  3153 2022-03-17        35        19     0     5   A     0     3   A   
        3175 2022-04-10        35         3     2     3   A     0     0   D   
        3198 2022-04-24        35         0     0     3   A     0     2   A   
        3214 2022-05-07        35        18     0     2   A     0     1   A   
        3237 2022-05-19        35        11     0     0   D     0     0   D   

                  Referee  ...  HST_rolling  AST_rolling  HF_rolling  \
H_Team                     ...                                         
Arsenal 60       A Taylor  ...     6.000000     6.333333   14.666667   
        89     C Kavanagh  ...     5.666667     5.000000   14.666667   
        100    A Marriner  ...     4.333333     4.000000   12.666667   
        116     S Attwell  ...     4.000000     3.333333    9.333333   
        137        M Dean  ...     4.333333     3.666667    8.666667   
...                   ...  ...          ...          ...         ...   
Wolves  3153     A Taylor  ...     5.333333    12.666667    7.333333   
        3175       J Moss  ...     6.000000    14.000000    7.333333   
        3198  N Swarbrick  ...     6.000000    12.333333    9.000000   
        3214    L Probert  ...     5.666667    12.000000    8.666667   
        3237      L Mason  ...     6.000000     9.000000    8.666667   

              AF_rolling  HC_rolling  AC_rolling  HY_rolling  AY_rolling  \
H_Team                                                                     
Arsenal 60     13.000000    5.666667    6.666667    1.666667    2.000000   
        89     14.000000    7.000000    5.666667    1.666667    2.000000   
        100    13.000000    5.666667    6.333333    2.000000    1.666667   
        116    11.333333    5.666667    6.000000    1.666667    1.666667   
        137    11.000000    7.333333    4.666667    1.666667    1.666667   
...                  ...         ...         ...         ...         ...   
Wolves  3153    9.666667    6.666667    5.333333    1.000000    1.666667   
        3175   10.666667    6.333333    6.333333    1.000000    1.000000   
        3198   10.666667    6.666667    5.000000    1.000000    1.333333   
        3214    8.666667    7.666667    5.000000    1.666667    0.666667   
        3237    8.333333    8.666667    4.000000    1.333333    0.666667   

              HR_rolling  AR_rolling  
H_Team                                
Arsenal 60      0.000000         0.0  
        89      0.000000         0.0  
        100     0.000000         0.0  
        116     0.000000         0.0  
        137     0.000000         0.0  
...                  ...         ...  
Wolves  3153    0.000000         0.0  
        3175    0.333333         0.0  
        3198    0.333333         0.0  
        3214    0.666667         0.0  
        3237    0.333333         0.0  

[3522 rows x 38 columns]

In [13]:
matches_rolling = matches_rolling.droplevel("H_Team")
matches_rolling

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HST_rolling,AST_rolling,HF_rolling,AF_rolling,HC_rolling,AC_rolling,HY_rolling,AY_rolling,HR_rolling,AR_rolling
60,2011-10-01,0,31,2,0,H,0,0,D,A Taylor,...,6.000000,6.333333,14.666667,13.000000,5.666667,6.666667,1.666667,2.000000,0.000000,0.0
89,2011-10-29,0,16,3,1,H,1,1,D,C Kavanagh,...,5.666667,5.000000,14.666667,14.000000,7.000000,5.666667,1.666667,2.000000,0.000000,0.0
100,2011-11-05,0,17,1,1,D,0,0,D,A Marriner,...,4.333333,4.000000,12.666667,13.000000,5.666667,6.333333,2.000000,1.666667,0.000000,0.0
116,2011-11-19,0,35,1,1,D,0,1,A,S Attwell,...,4.000000,3.333333,9.333333,11.333333,5.666667,6.000000,1.666667,1.666667,0.000000,0.0
137,2011-12-04,0,30,4,2,H,1,2,A,M Dean,...,4.333333,3.666667,8.666667,11.000000,7.333333,4.666667,1.666667,1.666667,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3153,2022-03-17,35,19,0,5,A,0,3,A,A Taylor,...,5.333333,12.666667,7.333333,9.666667,6.666667,5.333333,1.000000,1.666667,0.000000,0.0
3175,2022-04-10,35,3,2,3,A,0,0,D,J Moss,...,6.000000,14.000000,7.333333,10.666667,6.333333,6.333333,1.000000,1.000000,0.333333,0.0
3198,2022-04-24,35,0,0,3,A,0,2,A,N Swarbrick,...,6.000000,12.333333,9.000000,10.666667,6.666667,5.000000,1.000000,1.333333,0.333333,0.0
3214,2022-05-07,35,18,0,2,A,0,1,A,L Probert,...,5.666667,12.000000,8.666667,8.666667,7.666667,5.000000,1.666667,0.666667,0.666667,0.0


In [14]:
matches_rolling.index = range(matches_rolling.shape[0])
matches_rolling

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HST_rolling,AST_rolling,HF_rolling,AF_rolling,HC_rolling,AC_rolling,HY_rolling,AY_rolling,HR_rolling,AR_rolling
0,2011-10-01,0,31,2,0,H,0,0,D,A Taylor,...,6.000000,6.333333,14.666667,13.000000,5.666667,6.666667,1.666667,2.000000,0.000000,0.0
1,2011-10-29,0,16,3,1,H,1,1,D,C Kavanagh,...,5.666667,5.000000,14.666667,14.000000,7.000000,5.666667,1.666667,2.000000,0.000000,0.0
2,2011-11-05,0,17,1,1,D,0,0,D,A Marriner,...,4.333333,4.000000,12.666667,13.000000,5.666667,6.333333,2.000000,1.666667,0.000000,0.0
3,2011-11-19,0,35,1,1,D,0,1,A,S Attwell,...,4.000000,3.333333,9.333333,11.333333,5.666667,6.000000,1.666667,1.666667,0.000000,0.0
4,2011-12-04,0,30,4,2,H,1,2,A,M Dean,...,4.333333,3.666667,8.666667,11.000000,7.333333,4.666667,1.666667,1.666667,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3517,2022-03-17,35,19,0,5,A,0,3,A,A Taylor,...,5.333333,12.666667,7.333333,9.666667,6.666667,5.333333,1.000000,1.666667,0.000000,0.0
3518,2022-04-10,35,3,2,3,A,0,0,D,J Moss,...,6.000000,14.000000,7.333333,10.666667,6.333333,6.333333,1.000000,1.000000,0.333333,0.0
3519,2022-04-24,35,0,0,3,A,0,2,A,N Swarbrick,...,6.000000,12.333333,9.000000,10.666667,6.666667,5.000000,1.000000,1.333333,0.333333,0.0
3520,2022-05-07,35,18,0,2,A,0,1,A,L Probert,...,5.666667,12.000000,8.666667,8.666667,7.666667,5.000000,1.666667,0.666667,0.666667,0.0


In [15]:
predictors = ["day_code","HomeTeam","AwayTeam","HTHG","HTAG"]
predictors = np.concatenate((predictors, new_cols))
target = 'target'

In [16]:
predictors = ["day_code","HomeTeam","AwayTeam","HTHG","HTAG"]
# ,"HS","AS","HST","AST","HF","AF","HC","AC","HY","AY","HR","AR"
predictors = np.concatenate((predictors, new_cols))
target = "target"

train = matches_rolling[matches_rolling["Date"] < "2018-12-29"]
test = matches_rolling[matches_rolling["Date"] > "2018-12-29"]

# train = matches_rolling[matches_rolling["Date"] < "13-12-2018"]
# test = matches_rolling[matches_rolling["Date"] > "13-12-2018"]

scaler = StandardScaler()
train[predictors] = scaler.fit_transform(train[predictors])
test[predictors] = scaler.transform(test[predictors])

X_train = train[predictors].values.reshape((train.shape[0], len(predictors), 1))
X_test = test[predictors].values.reshape((test.shape[0], len(predictors), 1))
y_train = train[target].values
y_test = test[target].values

y_train_cat = pd.get_dummies(y_train)
y_test_cat = pd.get_dummies(y_test)

model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train_cat, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50


C:\Users\honey\AppData\Local\Temp\ipykernel_7708\2555538772.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[predictors] = scaler.fit_transform(train[predictors])
C:\Users\honey\AppData\Local\Temp\ipykernel_7708\2555538772.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[predictors] = scaler.transform(test[predictors])
c:\Users\honey\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to 

69/69 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.4548 - loss: 1.0737 - val_accuracy: 0.4392 - val_loss: 1.0601
Epoch 2/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4473 - loss: 1.0682 - val_accuracy: 0.4392 - val_loss: 1.0590
Epoch 3/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4578 - loss: 1.0596 - val_accuracy: 0.4392 - val_loss: 1.0761
Epoch 4/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.4599 - loss: 1.0608 - val_accuracy: 0.4392 - val_loss: 1.0613
Epoch 5/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4746 - loss: 1.0458 - val_accuracy: 0.4392 - val_loss: 1.0609
Epoch 6/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4814 - loss: 1.0457 - val_accuracy: 0.4392 - val_loss: 1.0584
Epoch 7/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4737 - loss: 1.0463 - val_accuracy: 0.4392 - val_loss: 1.0889
Epoch 8/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4701 - loss: 1.0536 - val_accuracy: 0.4392 - val_loss: 1.

In [17]:
_, accuracy = model.evaluate(X_test, y_test_cat)
print('ความแม่นยำ: %.2f' % (accuracy*100))

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6069 - loss: 0.8369
ความแม่นยำ: 60.50


In [18]:
predictions = model.predict(X_test)
y_pred= np.argmax(predictions, axis=1)
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


Predicted,0,1,2
Actual,,,
0,139,37,40
1,32,257,54
2,71,67,65


In [19]:
def averages_five_match_homeTeam(group, cols,new_cols):
    group = group.sort_values("Date")
    rolling_stats = group[cols].rolling(5,closed="left").mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    group = group.tail(1)
    group = group[["HomeTeam","HTHG_rolling",
               'HS_rolling',
               'HST_rolling',
               'HF_rolling',
               'HC_rolling',
               'HY_rolling',
               'HR_rolling']]

    return group

In [20]:
def averages_five_match_AwayTeam(group, cols,new_cols):
    group = group.sort_values("Date")
    rolling_stats = group[cols].rolling(5,closed="left").mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    group = group.tail(1)
    group = group[["AwayTeam","HTAG_rolling",
                          'AS_rolling',
                          'AST_rolling',
                          'AF_rolling',
                          'AC_rolling',
                          'AY_rolling',
                          'AR_rolling']]
    return group

In [21]:
# from tensorflow.keras.models import load_model
# model = load_model('football_predict.h5')

def predict_match_result(Day_Of_Week,HomeTeam, AwayTeam):
  df_matches_h = df.groupby("H_Team")
  group_h = df_matches_h.get_group(HomeTeam)
  cols_h = ["HTHG","HS","HST","HF","HC","HY","HR"]
  new_cols_h = [f"{c}_rolling" for c in cols_h]

  df_matches_a = df.groupby("A_Team")
  group_a = df_matches_a.get_group(AwayTeam)
  cols_a = ["HTAG","AS","AST","AF","AC","AY","AR"]
  new_cols_a = [f"{c}_rolling" for c in cols_a]

  data = averages_five_match_homeTeam(group_h,cols_h,new_cols_h)
  data = data.rename(columns = {"HTHG_rolling" : "HTHG"})
  data_a = averages_five_match_AwayTeam(group_a,cols_a,new_cols_a)
  data_a = data_a.rename(columns = {"HTAG_rolling" : "HTAG"})
  # data_a = data_a.rename(columns = {"HomeTeam" : "AwayTeam"})

  result = data.join(data_a, how="outer")
  result.fillna(result.shift(), inplace=True)
  if(len(result) >= 2):
    result.drop(result.index[0], inplace=True)
  result["day_code"] = Day_Of_Week
  result = result.reindex(columns=predictors)

  y_train_pred = model.predict(result)
  winner= np.argmax(y_train_pred, axis=1)

  return y_train_pred*100,winner,result
y_train_pred,winner,result = predict_match_result(4,"Man City","Liverpool")
y_train_pred   

ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None

In [ ]:
# model.save('football_predict_3.h5')

c:\Users\honey\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
